In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from split import acquire_train_test_data
import warnings
warnings.filterwarnings("ignore")

In [2]:
train,test=acquire_train_test_data()
train,validate=train_test_split(train, train_size=.8, random_state=123, stratify = train.hospital_death)

# Build a function without cross validation

In [6]:
y_train = train['hospital_death']
X_train = train.drop(columns='hospital_death')
y_validate = validate['hospital_death']
X_validate = validate.drop(columns='hospital_death')

In [14]:
dt = DecisionTreeClassifier(max_depth=10,criterion='gini')
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [15]:
dt_probs_tra = dt.predict_proba(X_train)
dt_probs_val = dt.predict_proba(X_validate)
dt_probs_tra = dt_probs_tra[:, 1]
dt_probs_val = dt_probs_val[:, 1]
score_tra= roc_auc_score(y_train, dt_probs_tra)
score_val= roc_auc_score(y_validate, dt_probs_val)
score = (score_tra, score_val)

In [16]:
score

(0.8997676263476869, 0.8212178565436399)

In [3]:
from sklearn.model_selection import GridSearchCV

In [4]:
dt= DecisionTreeClassifier()

In [7]:
# keys are names of hyperparams, values are a list of values to try for that hyper parameter
params = {
    'max_depth': range(10, 15),
    'criterion': ['gini', 'entropy']
}

# cv=4 means 4-fold cross-validation, i.e. k = 4
grid = GridSearchCV(dt, params, cv=5,scoring='roc_auc')
grid.fit(X_train, y_train)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(10, 15)},
             pre_dispat

In [8]:
grid.best_score_

0.7917895121847395

In [12]:
grid.best_params_

{'criterion': 'gini', 'max_depth': 10}

In [9]:
model = grid.best_estimator_
model.score(X_validate, y_validate)

0.9165190132206624

In [26]:
pred_prob = model.predict_proba(X_train)
pred_prob = pred_prob[:, 1]

In [27]:
roc_auc_score(y_train, pred_prob)

0.8995198766916273

In [23]:
pred_prob = model.predict_proba(X_validate)
pred_prob = pred_prob[:, 1]

In [25]:
roc_auc_score(y_validate, pred_prob)

0.8194217830386801

In [19]:
from sklearn.metrics import confusion_matrix

In [10]:
y_test = test['hospital_death']
X_test = test.drop(columns='hospital_death')

In [11]:
model.score(X_test, y_test)

0.9177342855585237